# What is Replication

<img src="replica-set-read-write-operations-primary.bakedsvg.svg">

### Automatic Failover (故障轉移)

<img src="replica-set-trigger-election.bakedsvg.svg">

### Arbiter

<img src="replica-set-primary-with-secondary-and-arbiter.bakedsvg.svg">

# Hidden Replica Set Members

<img src="replica-set-hidden-member.bakedsvg.svg">

### 部署奇數個成員

### 範例01 Setting Up a Replica Set

In [ ]:
storage:
  dbPath: /var/mongodb/db/node1
net:
  bindIp: 192.168.103.100,localhost
  port: 27011
security:
  authorization: enabled
  keyFile: /var/mongodb/pki/m103-keyfile
systemLog:
  destination: file
  path: /var/mongodb/db/node1/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-example

In [ ]:
sudo mkdir -p /var/mongodb/pki/
sudo chown vagrant:vagrant /var/mongodb/pki/
openssl rand -base64 741 > /var/mongodb/pki/m103-keyfile
chmod 400 /var/mongodb/pki/m103-keyfile

In [ ]:
mkdir -p /var/mongodb/db/node1

In [ ]:
mongod -f node1.conf

In [ ]:
cp node1.conf node2.conf
cp node2.conf node3.conf

In [ ]:
vi node2.conf

In [ ]:
:wq

In [ ]:
storage:
  dbPath: /var/mongodb/db/node2
net:
  bindIp: 192.168.103.100,localhost
  port: 27012
security:
  authorization: enabled
  keyFile: /var/mongodb/pki/m103-keyfile
systemLog:
  destination: file
  path: /var/mongodb/db/node2/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-example

In [ ]:
storage:
  dbPath: /var/mongodb/db/node3
net:
  bindIp: 192.168.103.100,localhost
  port: 27013
security:
  authorization: enabled
  keyFile: /var/mongodb/pki/m103-keyfile
systemLog:
  destination: file
  path: /var/mongodb/db/node3/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-example

In [ ]:
mkdir /var/mongodb/db/{node2,node3}

In [ ]:
mongod -f node2.conf
mongod -f node3.conf

In [ ]:
mongo --port 27011

In [ ]:
rs.initiate()

In [ ]:
use admin
db.createUser({
  user: "m103-admin",
  pwd: "m103-pass",
  roles: [
    {role: "root", db: "admin"}
  ]
})

In [ ]:
exit
mongo --host "m103-example/192.168.103.100:27011" -u "m103-admin"
-p "m103-pass" --authenticationDatabase "admin"

In [ ]:
rs.status()

In [ ]:
rs.add("m103:27012")
rs.add("m103:27013")

In [ ]:
rs.isMaster()

In [ ]:
rs.stepDown()

In [ ]:
rs.isMaster()

# Replication Configuration Document

### 簡單結構介紹

# Replication Commands

<img src="7.PNG">

<img src="8.PNG">

<img src="9.PNG">

<img src="10.PNG">

<img src="11.PNG">

<img src="12.png">

# Local DB

In [ ]:
mkdir allbymyselfdb
mongod --dbpath allbymyselfdb

In [ ]:
mongo
show dbs

In [ ]:
use local
show collections

In [ ]:
use local
db.oplog.rs.find()

In [ ]:
var stats = db.oplog.rs.stats()

In [ ]:
stats.capped

In [ ]:
stats.size

In [ ]:
stats.maxSize

---

In [ ]:
# 換單位成 MB
# by default will take 5% of yout free disk
var stats = db.oplog.rs.stats(1024*1024)
stats.maxSize

In [ ]:
df -h

In [ ]:
re.printReplicationInfo()

---

In [ ]:
use m103
db.createCollection('messages')

In [ ]:
use local
db.oplog.rs.find( { "o.msg": { $ne: "periodic noop" } } ).sort( { $natural: -1 } ).limit(1).pretty()

In [ ]:
use m103
for ( i=0; i< 100; i++) { db.messages.insert( { 'msg': 'not yet', _id: i } ) }
db.messages.count()

In [ ]:
use local
db.oplog.rs.find({"ns": "m103.messages"}).sort({$natural: -1})

In [ ]:
use m103
db.messages.updateMany( {}, { $set: { author: 'norberto' } } )
use local
db.oplog.rs.find( { "ns": "m103.messages" } ).sort( { $natural: -1 } )

# Reconfiguring a Running Replica Set

<img src="13.png">

In [ ]:
要再多增加一個 secondary 和一個 arbiter

<img src="14.png">

In [ ]:
# 去看看現在 replica set 的 topology
rs.isMaster()

In [ ]:
# node4.conf
storage:
  dbPath: /var/mongodb/db/node4
net:
  bindIp: 192.168.103.100,localhost
  port: 27014
systemLog:
  destination: file
  path: /var/mongodb/db/node4/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-example

In [ ]:
# arbiter.conf
storage:
  dbPath: /var/mongodb/db/arbiter
net:
  bindIp: 192.168.103.100,localhost
  port: 28000
systemLog:
  destination: file
  path: /var/mongodb/db/arbiter/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-example

In [ ]:
mongod -f node4.conf
mongod -f arbiter.conf

In [ ]:
rs.add("m103:27014")
rs.addArb("m103:28000")

In [ ]:
rs.isMaster()

In [ ]:
rs.remove("m103:28000")

In [ ]:
# 但現在我們只有 4 個成員
rs.isMaster()

In [ ]:
cfg = rs.conf()

In [ ]:
cfg

In [ ]:
cfg.members[3].votes = 0
cfg.members[3].hidden = true
cfg.members[3].priority = 0

In [ ]:
# 會引發一場投票
rs.reconfig(cfg)

In [ ]:
rs.conf()

<img src="15.png">

# Reads and Writes on a Replica Set

In [ ]:
mongo --host "m103-example/m103:27011" -u "m103-admin" -p
"m103-pass" --authenticationDatabase "admin"

In [ ]:
rs.isMaster()

In [ ]:
use newDB
db.new_collection.insert( { "student": "Matt Javaly", "grade": "A+" } )

In [ ]:
# 前面不能加 replica set 的名字才能到 secondary
mongo --host "m103:27012" -u "m103-admin" -p "m103-pass"
--authenticationDatabase "admin"

In [ ]:
show dbs

In [ ]:
rs.slaveOk()

In [ ]:
# write comment 有成功被複寫到 secondary node
use newDB
db.new_collection.find()

In [ ]:
db.new_collection.insert( { "student": "Norberto Leite", "grade": "B+" } )

---

In [ ]:
use admin
db.shutdownServer()

In [ ]:
mongo --host "m103:27011" -u "m103-admin" -p "m103-pass"
--authenticationDatabase "admin"

In [ ]:
rs.isMaster()

# Failover and Elections

### 範例01

In [ ]:
cfg = rs.conf()

In [ ]:
cfg.members[2].priority = 0

In [ ]:
rs.reconfig(cfg)

In [ ]:
rs.isMaster()

In [ ]:
rs.stepDown()

In [ ]:
rs.isMaster()

# Write Concerns

<img src="16.PNG">

### Write Concern Levels

### Write Concern Options

# Read Concerns

### Read Concern Levels

<img src="19.PNG">

# Read Preferences

<img src="17.PNG">

### Read Preference Models

<img src="18.PNG">

### 範例01